# <center> *PyWAsP* tutorial 5 <br><br> Importing objects from WWH file <center>


In this tutorial we will cover a `pywasp` feature that lets users import objects from `WAsP` workspace file (`wwh` file) and perform `pywasp` calculations with them. We will use an example of 'Parque Ficticio' workspace, which is a common example from `WAsP` courses. For this location we will generate site effectes and predicted wind climate over a uniform grid.

First let's import necessary libraries and read WAsP Workspace file (`wwh`) and list its content:

In [1]:
import warnings
warnings.filterwarnings('ignore')  # We will ignore warnings to avoid cluttering the notebook

import pywasp as pw
import numpy as np

In [2]:
wwh = pw.io.Workspace.read_wwh('./data/import/ParqueFicticio.wwh')
wwh

 Object ID        Object description
         3                Vector map
         4        Turbine site group
        15  Generalised wind climate
        17     Observed wind climate
        20                Vector map

The listed objects can be imported as `pywasp` `xarray` datasets by calling specifing `geter` methods and suppling  the desired object `id` and depending on the object type also some additional parameters. First thing to notice is that we have two ids assigned to vector map objects. These two vector maps are identical.  The reason why there two ids/ two maps is that each id tells `WAsP` to read the vector map differently, one as an `elevation` map while the second as a `roughness` map. As you will see later in this tutorial we do this a bit different in `pywasp` comparing to `WAsP` GUI. 

Let's now start importing listed objects and as an exercise exporting them as `NetCDF` files.
Let's start with `Observed Wind Climate`. To get the `OWC`, we call method `get_owc()` and provide id of this object:

In [3]:
bwc = wwh.get_owc(17)
print(bwc)

<xarray.Dataset>
Dimensions:      (point: 1, sector: 12, wsbin: 27)
Coordinates:
  * wsbin        (wsbin) float64 0.5 1.5 2.5 3.5 4.5 ... 23.5 24.5 25.5 26.5
  * sector       (sector) float64 0.0 30.0 60.0 90.0 ... 240.0 270.0 300.0 330.0
    south_north  (point) float64 -31.5
    west_east    (point) float64 -71.5
    height       (point) float64 42.25
    wsceil       (wsbin) float64 1.0 2.0 3.0 4.0 5.0 ... 24.0 25.0 26.0 27.0
    crs          int64 0
Dimensions without coordinates: point
Data variables:
    wdfreq       (sector, point) float64 0.0706 0.0318 0.04788 ... 0.1284 0.0883
    wsfreq       (wsbin, sector, point) float64 0.02467 0.01616 ... 0.0002156
    amplif       float64 1.0
    offset       float64 0.0
Attributes:
    header:   Cerro


In [4]:
bwc.to_netcdf('./data/export/bwc.nc')

In a similar fashion we are importing `Generalised wind climate`, and exporting it to `NetCDF` file:

In [5]:
gwc = wwh.get_gwc(15)
print(gwc)

gwc.to_netcdf('./data/export/gwc.nc')

<xarray.Dataset>
Dimensions:        (gen_height: 5, gen_roughness: 5, point: 1, sector: 12)
Coordinates:
  * gen_roughness  (gen_roughness) float64 0.0 0.03 0.1 0.4 1.5
  * gen_height     (gen_height) float64 10.0 25.0 50.0 100.0 200.0
  * sector         (sector) float64 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
    south_north    (point) float64 -31.5
    west_east      (point) float64 -71.5
    height         (point) float64 42.25
    crs            int64 0
Dimensions without coordinates: point
Data variables:
    wdfreq         (sector, gen_roughness, point) float64 0.009726 ... 0.02108
    A              (sector, gen_height, gen_roughness, point) float64 4.515 ... 5.945
    k              (sector, gen_height, gen_roughness, point) float64 1.83 ... 1.775
Attributes:
    header:   Cerro


Now let's load the maps.  To do this we call method `get_vectormap` which requires following parameters:
 - `id`: id of the vector map object in the wwh list (integer)
 - `srs`: EPSG code of the map
 - `map_type`: feature type to extract from mapfile, it can be 'elevation' or 'roughness'

In our example the EPSG code is `32719`, since the vector map header has following information:
```
UTM-Proj.-S.hemisph. Zone 19 (WGS 1984) 

```

We will extract both the elevation and roughness map, and we can supply any of the two `id`s (3 or 20). 
We will then boundle these two maps creating topo map which is needed for the follow up calculations.

In [7]:
elev_map = wwh.get_vectormap(20, 32719, "elevation")
rough_map, lut = wwh.get_vectormap(20, 32719, "roughness")

topo_map = pw.TopographyMap(elev_map, rough_map, lut)

If we closely inspect `bwc` data we will notice that the provided geospatial location of the mast in this dataset is in the projected coordinates of the EPSG:4326 projection (i.e., south_north=latitude and west_east=longitude). However, as our terrain and rougness data are in the South UTM Zone 19 (EPSG:32719) we will convert and update the mast coordinates. Once the coordiantes are converted we will store them in in variables `loc_x` and `loc_y` for later use

In [8]:
bwc = pw.gis_tools.reproject_ds(bwc, 32719) #
loc_y = bwc.south_north
loc_x = bwc.west_east

Let's now create uniform grid, with resolution of 100 m in `west_east` and `north_south` coordinate. The grid will be positioned 200 m above the ground level. We use `pw.create_dataset` to perform this.

In [9]:
height = 200
x_res = 100
y_res = 100

output_locs = pw.create_dataset(
    np.arange(262878, 265078 + x_res, x_res),
    np.arange(6504214, 6507414 + y_res, y_res),
    np.array([height]),
    32719
)

Let's now calculate predicted wind climate,  site effects, and meteorological variables, using the pw.wasp.downscale routine. We will then export this to a `NetCDF` file:

In [10]:
conf = pw.Config()
pwc = pw.wasp.downscale(gwc, topo_map, output_locs, conf, genwc_interp='nearest', return_site_factors=True)
print(pwc)
pwc.to_netcdf('./data/export/results_'+str(height)+'_m.nc')

<xarray.Dataset>
Dimensions:              (height: 1, sector: 12, south_north: 33, west_east: 23)
Coordinates:
    crs                  int64 0
  * sector               (sector) float64 0.0 30.0 60.0 ... 270.0 300.0 330.0
  * height               (height) int64 200
  * south_north          (south_north) int64 6504214 6504314 ... 6507314 6507414
  * west_east            (west_east) int64 262878 262978 ... 264978 265078
Data variables:
    z0meso               (sector, height, south_north, west_east) float32 0.099999994 ... 0.086560346
    slfmeso              (sector, height, south_north, west_east) float32 1.0 ... 0.97677594
    displ                (sector, height, south_north, west_east) float32 0.0 ... 0.0
    user_def_speedups    (sector, height, south_north, west_east) float32 1.0 ... 1.0
    orographic_speedups  (sector, height, south_north, west_east) float32 0.9471683 ... 1.0982621
    obstacle_speedups    (sector, height, south_north, west_east) float32 1.0 ... 1.0
    roughne

Finally lets extract positions of wind turbines and their hub height. To do this we use method `get_turbines()`. This method requires `id` and `srs` as inputs and generated an `xarray` dataset with positions of wind turbines:

In [16]:
wtg_pos = wwh.get_turbines(4, 32719)
print(wtg_pos)

<xarray.Dataset>
Dimensions:      (point: 8)
Coordinates:
    height       (point) float64 70.0 70.0 70.0 70.0 70.0 70.0 70.0 70.0
    south_north  (point) float64 6.505e+06 6.505e+06 ... 6.506e+06 6.507e+06
    west_east    (point) float64 2.639e+05 2.64e+05 ... 2.639e+05 2.637e+05
    crs          int64 0
Dimensions without coordinates: point
Data variables:
    output       (point) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
